In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import time
import os
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, StratifiedShuffleSplit, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, f_classif, chi2
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, confusion_matrix
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib

from scipy import stats
import scipy.io
import pyriemann as pr
import networkx as nx

import warnings
warnings.filterwarnings("ignore")

In [1]:
class SPD_pipeline():
    def __init__(self, threshold, lam):
        self.threshold = threshold
        self.lam = lam
        
    def get_spd_dataset(self, data):
        res = map(self.create_spd, data)
        return np.stack(res, axis=0)
        
    def create_spd(self, X):
        N = X.shape[0]
        binary_matrix = (abs(X) > self.threshold).astype(int)
#         binary_matrix = X
        G = nx.from_numpy_array(binary_matrix)
        X_lp = nx.laplacian_matrix(G)
        return X_lp.toarray() + self.lam*np.eye(N)
        
    def get_svc_grid(self, cv, scoring, random_state=None, n_jobs=1,
                     svc_kernel_l=None, svc_c_l=None, svc_gamma_l=None, metric=None, update = None):

        """
        define GridSearchCV object for kernel SVC pipeline, for given parameters
        """
        clf = pr.classification.TSclassifier(clf=SVC(random_state=random_state, probability=True))

        param_grid = { }
        
        if svc_kernel_l is not None:
            param_grid['clf__kernel'] = svc_kernel_l
        if svc_c_l is not None:
            param_grid['clf__C'] = svc_c_l
        if svc_gamma_l is not None:
            param_grid['clf__gamma'] = svc_gamma_l
        if metric is not None:
            param_grid['metric'] = metric
        if update is not None:
            param_grid['tsupdate'] = update
        
        return GridSearchCV(estimator = clf, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=n_jobs, verbose = 1)
    
    def get_lr_grid(self, cv, scoring, random_state=None, n_jobs=1, lr_c_l=None, metric=None, update=None):
        
        """
        define GridSearchCV object for LR on tangency plane pipeline
        """

        clf = pr.classification.TSclassifier(clf=LogisticRegression(random_state=random_state))
        param_grid = { }
        
        if lr_c_l is not None:
            param_grid['clf__C'] = lr_c_l
        if metric is not None:
            param_grid['metric'] = metric
        if update is not None:
            param_grid['tsupdate'] = update
        
        return GridSearchCV(estimator=clf, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=n_jobs, verbose = 1)
    
    def get_knn_grid(self, cv, scoring, random_state=None, n_jobs=1, knn_n_neighbors_l=None, metric=None):
        """
        define GridSearchCV object for kNN on the manifold pipeline
        """
        clf = pr.classification.KNearestNeighbor(n_jobs=n_jobs)
        param_grid = { }
        
        if knn_n_neighbors_l is not None:
            param_grid['n_neighbors'] = knn_n_neighbors_l
        if metric is not None:
            param_grid['metric'] = metric
            
        return  GridSearchCV(estimator=clf, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=n_jobs, verbose = 1)

            
    def get_mdm_grid(self, cv, scoring, random_state=None, n_jobs=1, metric=None):
        """
        define GridSearchCV object for min distance to mean on the manifold pipeline
        """

        clf = pr.classification.MDM(n_jobs=n_jobs)
        
        param_grid = { }
        
        if metric is not None:
            param_grid['metric'] = metric
            
        return  GridSearchCV(estimator=clf, param_grid=param_grid, scoring=scoring, cv=cv, n_jobs=n_jobs, verbose = 1)
    
    
    def train_grid_cv(self, data, y, n_splits, n_repeats, 
                      scoring, pos_label=None, random_state=None, 
                      n_jobs=1, save_plot_to=None, save_models_to=None):
                     
        """
        train all the grids, passed to it
        returns: best model and all the trained grids
        """
        X = self.get_spd_dataset(data)

#         X = data 
        cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
        print("Target distribution: ")
        print(y.value_counts(), "\n")
        if pos_label is None:
            y_enc = pd.Series(LabelEncoder().fit_transform(y), index=y.index)
        else:
            y_enc = pd.Series(y == pos_label, dtype=int)

        #### SVC ####
        if os.path.isfile(save_models_to + '_svc.pkl'):
            print('loading fitted SVC...\n')
            grid_cv_svc = self.load_model(save_models_to + '_svc.pkl')
        else:
            print("Training SVC...")
            grid_cv_svc = self.get_svc_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
                                       svc_kernel_l=["rbf", "linear"],
                                       svc_c_l=[10 ** i for i in range(-4, 4, 2)],
                                       svc_gamma_l=[10 ** i for i in range(-4, 2, 2)],
                                       metric = ['riemann', 'logeuclid', 'euclid'])
#             grid_cv_svc = self.get_svc_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
#                                        svc_kernel_l=["linear"],
#                                        svc_c_l=[10 ** i for i in range(-1, 1, 2)],
#                                        svc_gamma_l=[10 ** i for i in range(-1, 1, 2)],
#                                        metric = ['riemann'])
            start_time = time.time()
            grid_cv_svc.fit(X, y_enc)
            print("(training took {}s)\n".format(time.time() - start_time))
            
        if save_models_to is not None:
            self.save_model(grid_cv_svc, save_models_to + '_svc.pkl')
            
        #### LR ####    
        if os.path.isfile(save_models_to + '_lr.pkl'):
            print('loading fitted LR...\n')
            grid_cv_lr = self.load_model(save_models_to + '_lr.pkl')
        else:
            print("Training LR...")
            grid_cv_lr = self.get_lr_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
                                     lr_c_l=[10 ** i for i in range(-4, 2, 2)],
                                     metric = ['riemann', 'logeuclid', 'euclid'])
#             grid_cv_lr = self.get_lr_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
#                                      lr_c_l=[10 ** i for i in range(-1, 1, 2)],
#                                      metric = ['riemann'])
            start_time = time.time()
            grid_cv_lr.fit(X, y_enc)
            print("(training took {}s)\n".format(time.time() - start_time))
            
        if save_models_to is not None:
            self.save_model(grid_cv_lr, save_models_to + '_lr.pkl')
            
        #### KNN ####    
        if os.path.isfile(save_models_to + '_knn.pkl'):
            print('loading fitted KNN...\n')
            grid_cv_knn = self.load_model(save_models_to + '_knn.pkl')
        else:
            print("Training KNN...")
            class_size_tr = min(y.value_counts())
            grid_cv_knn = self.get_knn_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
                                       knn_n_neighbors_l=[i for i in range(5, class_size_tr - 1, 5)],
                                       metric = ['riemann', 'logeuclid', 'euclid'])
#             grid_cv_knn = self.get_knn_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
#                                        knn_n_neighbors_l=[10],
#                                        metric = ['riemann'])        
            start_time = time.time()
            grid_cv_knn.fit(X, np.array(y_enc))
            print("(training took {}s)\n".format(time.time() - start_time))
            
        if save_models_to is not None:
            self.save_model(grid_cv_knn, save_models_to + '_knn.pkl')
            
            
        #### MDM ####    
        if os.path.isfile(save_models_to + '_mdm.pkl'):
            print('loading fitted MDM...\n')
            grid_cv_mdm = self.load_model(save_models_to + '_mdm.pkl')
        else:
            print("Training MDM...")
            grid_cv_mdm = self.get_mdm_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
                                       metric = ['riemann', 'logeuclid', 'euclid'])
#             grid_cv_mdm = self.get_mdm_grid(cv, scoring, random_state=random_state, n_jobs=n_jobs,
#                                        metric = ['riemann'])
            start_time = time.time()
            grid_cv_mdm.fit(X, y_enc)
            print("(training took {}s)\n".format(time.time() - start_time))

        if save_models_to is not None:
            self.save_model(grid_cv_mdm, save_models_to + '_mdm.pkl')
            
        
        print("Scoring:", scoring)
        self.print_results({
            "SVC" : grid_cv_svc,
            "LR" : grid_cv_lr,
            "KNN" : grid_cv_knn,
            "MDM" : grid_cv_mdm
                      }, save_plot_to=save_plot_to)

        best_model = max([grid_cv_svc, grid_cv_lr, grid_cv_mdm, grid_cv_knn], key=lambda x: x.best_score_).best_estimator_

        return best_model, grid_cv_svc, grid_cv_lr, grid_cv_mdm, grid_cv_knn
    
    def print_results(self, clf_grid_dict, save_plot_to=None):
        """
        Draw beautiful table with results
        """
        results = {
                "classifier" : [], 
                "best parameters" : [],
                "mean" : [],
                "std" : []
               }

        for clf, grid in clf_grid_dict.items():
            results["classifier"].append(clf)
            results["best parameters"].append(", ".join(
                [param + " = " + str(best_value) for param, best_value in grid.best_params_.items()]))
#                 results["best dim. reduction method"].append(grid.best_params_['dim_reduction'])
            idx = grid.best_index_
            results["mean"].append(grid.cv_results_['mean_test_score'][idx])
            results["std"].append(grid.cv_results_['std_test_score'][idx])

        results = pd.DataFrame(results, columns=["classifier", "best parameters", "mean", "std"])
        display(results.set_index("classifier"))

        # draw graph
        width = 0.9
        for i in results.index:
            plt.bar(i, results.loc[i, "mean"], width, yerr=results.loc[i, "std"], label=results.loc[i, "classifier"])
        plt.xticks(range(results.shape[0]), results.loc[:, "classifier"])
        plt.axis(ymin=0.0, ymax=1.0)
        if save_plot_to is not None:
            plt.savefig(save_plot_to)
        plt.show()

        print("Best model: ")
        clf = results.loc[results["mean"].argmax(), "classifier"]
        print(clf)
        print("\n".join(
                [param + " = " + str(best_value) for param, best_value in clf_grid_dict[clf].best_params_.items()]))
    #     print("mean =", results["mean"].max())
    #     print("std =", results.loc[results["mean"].argmax(), "std"])
    n_objects = 100

    def repeated_cross_val_predict_proba(self, estimator, data, y, cv, pos_label=None, file=None):
        X = self.get_spd_dataset(data)
#         X = data
        n_objects = X.shape[0]
        
        if pos_label is None:
            y_enc = pd.Series(LabelEncoder().fit_transform(y), index=y.index)
        else:
            y_enc = pd.Series(y == pos_label, dtype=int)
        predictions = [[] for i in range(n_objects)]
        for idx_tr, idx_te in tqdm_notebook(cv.split(X, y_enc)):
            estimator.fit(X[idx_tr,:,:], y_enc.iloc[idx_tr])
            pred_te = np.array(estimator.predict_proba(X[idx_te,:,:]), dtype=float)
            for i, idx in enumerate(idx_te):
                predictions[idx].append(pred_te[i, 1])

        predictions = pd.DataFrame(predictions)
        if file is not None:
            predictions.to_csv(file)

        return predictions
    
    def plot_roc_curve(self, y, probas, pos_label, idx, average_repeats=False):
        if average_repeats:
            y_true = pd.Series(y == pos_label, dtype=int)
#             y_score = probas[idx].mean(axis=1)
            y_score = probas.mean(axis=1)
        else:
            n_repeats = probas.shape[1]
            y_true = pd.Series(np.tile(y == pos_label, (n_repeats)), dtype=int)
#             y_score = probas[idx].values.T.reshape(-1, 1)
            y_score = probas.values.T.reshape(-1, 1)
        fpr, tpr, t = roc_curve(y_true=y_true, y_score=y_score)
        plt.figure(figsize=(12, 8))
        plt.plot(fpr, tpr)
        plt.xlabel("False Positive rate", fontsize=14)
        plt.ylabel("True Positive rate", fontsize=14)
        plt.show()
        print("auc =", roc_auc_score(y_true, y_score))
        return fpr, tpr, t
    
    def get_fpr_fnr(self, fpr, tpr, fix_fpr_l=[0.1, 0.15, 0.2, 0.3]):
        fnr_l = []
        for fix_fpr in fix_fpr_l:
            fnr_l.append(1 - tpr[fpr <= fix_fpr][-1])
        fpr_fnr_table = pd.DataFrame(np.column_stack((fix_fpr_l, fnr_l)), columns=["False Positive rate (fixed)", "False Negative rate"])
        display(fpr_fnr_table)
        return fpr_fnr_table
    
    def save_model(self, model, file):
        joblib.dump(model, file)
    
    def load_model(self, file):
        model = joblib.load(file)
        return model